# Lesson 8 - Creating an A2A Healthcare Provider Agent using LangGraph and MCP

In this lesson, you will build a third agent: a Healthcare Provider Agent. This agent demonstrates a powerful combination of technologies:
1.  **MCP (Model Context Protocol)**: You will build a server that exposes a tool to find doctors from a JSON file.
2.  **LangGraph**: You will build an agent that uses this MCP tool.
3.  **A2A**: You will wrap the LangGraph agent in an A2A server.

## 8.1. Create the MCP Server

First, you will define an MCP server using `FastMCP`. This server exposes a tool called `list_doctors` which queries a local `doctors.json` file.

In [ ]:
%%writefile mcpserver.py
import json
from pathlib import Path

from mcp.server.fastmcp import FastMCP

# Initialize the server
mcp = FastMCP("doctorserver")

# Load Data
doctors: list = json.loads(Path("../data/doctors.json").read_text())


@mcp.tool()
def list_doctors(state: str | None = None, city: str | None = None) -> list[dict]:
    """This tool returns a list of doctors practicing in a specific location. The search is case-insensitive.

    Args:
        state: The two-letter state code (e.g., "CA" for California).
        city: The name of the city or town (e.g., "Boston").

    Returns:
        A JSON string representing a list of doctors matching the criteria.
        If no criteria are provided, an error message is returned.
        Example: '[{"name": "Dr John James", "specialty": "Cardiology", ...}]'
    """
    # Input validation: ensure at least one search term is given.
    if not state and not city:
        return [{"error": "Please provide a state or a city."}]

    target_state = state.strip().lower() if state else None
    target_city = city.strip().lower() if city else None

    return [
        doc
        for doc in doctors
        if (not target_state or doc["address"]["state"].lower() == target_state)
        and (not target_city or doc["address"]["city"].lower() == target_city)
    ]


# Kick off server if file is run
if __name__ == "__main__":
    mcp.run(transport="stdio")


## 8.2. Test with LangGraph and MCP Client

Now you will verify that you can connect to the MCP server and use it within a LangChain/LangGraph agent. You use `MultiServerMCPClient` to connect to the local script via `stdio` transport.

In [ ]:
import asyncio

from IPython.display import Markdown, display
from langchain.agents import create_agent
from langchain_mcp_adapters.client import MultiServerMCPClient
from langchain_mcp_adapters.sessions import StdioConnection
from langchain_openai import ChatOpenAI

from helpers import authenticate

In [ ]:
credentials, project_id = authenticate()

In [ ]:
import os 

location = "us-central1"
base_url = f"{os.getenv('GOOGLE_VERTEX_BASE_URL')}/v1/projects/{project_id}/locations/{location}/endpoints/openapi"

**Note:** The `base_url` looks a bit different from the one used in the video. This is because it was adapted to work in this learning environment. If you'd like to run the notebooks locally, make sure to check the [course repo](https://github.com/holtskinner/A2AWalkthrough/tree/main).

In [ ]:
import nest_asyncio

nest_asyncio.apply()

In [ ]:
mcp_client = MultiServerMCPClient(
    {
        "find_healthcare_providers": StdioConnection(
            transport="stdio",
            command="uv",
            args=["run", "mcpserver.py"],
        )
    }
)
tools = asyncio.run(mcp_client.get_tools())

In [ ]:
agent = create_agent(
    ChatOpenAI(
        model="openai/gpt-oss-20b-maas",
        openai_api_key=credentials.token,
        openai_api_base=base_url,
    ),
    tools,
    name="HealthcareProviderAgent",
    system_prompt="""Your task is to find and list healthcare providers 
    using the find_healthcare_providers MCP Tool based on the users query. 
    Only use providers based on the response from the tool.""",
)

In [ ]:
prompt = "I'm based in Austin, TX. Are there any Psychiatrists near me?"
response = await agent.ainvoke(
    {
        "messages": [
            {
                "role": "user",
                "content": prompt,
            }
        ]
    }
)

In [ ]:
display(Markdown(response["messages"][-1].content))

## 8.3. Define the Provider Agent Class

You will now encapsulate this logic into a `ProviderAgent` class and append it to `agents.py`. This class handles the MCP client connection and agent initialization.


In [ ]:
%%writefile -a agents.py
import os

from dotenv import load_dotenv
from langchain.agents import create_agent
from langchain_mcp_adapters.client import MultiServerMCPClient
from langchain_mcp_adapters.sessions import StdioConnection
from langchain_openai import ChatOpenAI

from helpers import authenticate

class ProviderAgent:
    def __init__(self) -> None:
        credentials, project_id = authenticate()
        location = "us-central1"
        base_url = f"{os.getenv('GOOGLE_VERTEX_BASE_URL')}/v1/projects/{project_id}/locations/{location}/endpoints/openapi"
        
        self.mcp_client = MultiServerMCPClient(
            {
                "find_healthcare_providers": StdioConnection(
                    transport="stdio",
                    command="uv",
                    args=["run", "mcpserver.py"],
                )
            }
        )

        self.credentials = credentials
        self.base_url = base_url
        self.agent = None

    async def initialize(self):
        """Initialize the agent asynchronously."""
        tools = await self.mcp_client.get_tools()
        self.agent = create_agent(
            ChatOpenAI(
                model="openai/gpt-oss-20b-maas",
                openai_api_key=self.credentials.token,
                openai_api_base=self.base_url,
            ),
            tools,
            name="HealthcareProviderAgent",
            system_prompt="""Your task is to find and list providers using 
            the find_healthcare_providers MCP Tool based on the users query. 
            Only use providers based on the response from the tool. Output 
            the information in a table.""",
        )
        return self

    async def answer_query(self, prompt: str) -> str:
        if self.agent is None:
            raise RuntimeError("""Agent not initialized. Call initialize() 
            first.""")

        response = await self.agent.ainvoke(
            {
                "messages": [
                    {
                        "role": "user",
                        "content": prompt,
                    }
                ]
            }
        )
        return response["messages"][-1].content

## 8.4. Test the Class

Verify the class works as expected before wrapping it in A2A.

In [ ]:
from agents import ProviderAgent

agent = await ProviderAgent().initialize()
result = await agent.answer_query(
    "I'm based in Austin, TX. Are there any Psychiatrists near me?"
)

display(Markdown(result))

## 8.5. Wrap in A2A Server

Finally, create the `a2a_provider_agent.py` server file. This uses the standard A2A wrapping pattern you learned in Lesson 3, but with the added complexity of asynchronous initialization for the MCP client in the `ProviderAgentExecutor`.

In [ ]:
%%writefile ../a2a_provider_agent.py
from dotenv import load_dotenv
import os
import uvicorn
from a2a.server.agent_execution import AgentExecutor, RequestContext
from a2a.server.apps import A2AStarletteApplication
from a2a.server.events import EventQueue
from a2a.server.request_handlers import DefaultRequestHandler
from a2a.server.tasks import InMemoryTaskStore
from a2a.types import (
    AgentCapabilities,
    AgentCard,
    AgentSkill,
)
from a2a.utils import new_agent_text_message
from agents import ProviderAgent

class ProviderAgentExecutor(AgentExecutor):
    """This is an agent for finding healthcare providers based on location and specialty."""
    
    def __init__(self) -> None:
        # Don't await in __init__ - it's not async
        self.agent = None
    
    async def _ensure_initialized(self) -> None:
        """Lazy initialization of the agent."""
        if self.agent is None:
            self.agent = await ProviderAgent().initialize()
    
    async def execute(
        self,
        context: RequestContext,
        event_queue: EventQueue,
    ) -> None:
        await self._ensure_initialized()
        
        prompt = context.get_user_input()
        response = await self.agent.answer_query(prompt)
        await event_queue.enqueue_event(new_agent_text_message(response))
    
    async def cancel(self, context: RequestContext, event_queue: EventQueue) -> None:
        pass

def main():
    print("Running Healthcare Provider Agent")
    load_dotenv()
    
    HOST = os.environ.get("AGENT_HOST", "localhost")
    PORT = int(os.environ.get("PROVIDER_AGENT_PORT", 9997))
    
    skill = AgentSkill(
        id="find_healthcare_providers",
        name="Find Healthcare Providers",
        description="Finds and lists healthcare providers based on user's location and specialty.",
        tags=["healthcare", "providers", "doctor", "psychiatrist"],
        examples=[
            "Are there any Psychiatrists near me in Boston, MA?",
            "Find a pediatrician in Springfield, IL.",
        ],
    )
    
    agent_card = AgentCard(
        name="HealthcareProviderAgent",
        description="An agent that can find and list healthcare providers based on a user's location and desired specialty.",
        url=f"http://{HOST}:{PORT}/",
        version="1.0.0",
        default_input_modes=["text"],
        default_output_modes=["text"],
        capabilities=AgentCapabilities(streaming=False),
        skills=[skill],
    )
    
    request_handler = DefaultRequestHandler(
        agent_executor=ProviderAgentExecutor(),
        task_store=InMemoryTaskStore(),
    )
    
    server = A2AStarletteApplication(
        agent_card=agent_card,
        http_handler=request_handler,
    )
    
    uvicorn.run(server.build(), host=HOST, port=PORT)
    
if __name__ == "__main__":
    main()

## 8.6. Run the Provider Server

Activate the Provider Agent in Terminal 3:
- Open Terminal 3 by running the cell below.
- Type `uv run a2a_provider_agent.py`

In [ ]:
import os

from IPython.display import IFrame

url = os.environ.get("DLAI_LOCAL_URL").format(port=8888)
IFrame(f"{url}terminals/3", width=800, height=600)

## 8.7. Resources

- [Model Context Protocol (MCP)](https://modelcontextprotocol.io/)
- [LangChain MCP Adapters](https://docs.langchain.com/oss/python/langchain/mcp)
- [Equivalent notebook in the course repo](https://github.com/holtskinner/A2AWalkthrough/blob/main/6_A2AxMCPLangGraph.ipynb)


<div style="background-color:#fff6ff; padding:13px; border-width:3px; border-color:#efe6ef; border-style:solid; border-radius:6px">
<p> ⬇ &nbsp; <b>Download Notebooks:</b> 1) click on the <em>"File"</em> option on the top menu of the notebook and then 2) click on <em>"Download"</em>.</p>
</div>